In [1]:
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
# from agents.dqn.agent import Agent, Config
# from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Basic constants and setting up environment

In [2]:
scenario = "convergence"

simTime = 10 # seconds
stepTime = 0.01  # seconds
history_length = 10


EPISODE_COUNT = 20
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": 10,
    "agentType": Agent.TYPE,
    "nonZeroStart": True,
    "scenario": "convergence",
    "nWifi": 7
}
print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

Steps per episode: 1000


In [3]:
# config = Config(buffer_size=2e3, batch_size=64, gamma=0.99, tau=1e-3, lr=5e-4)
config = Config(buffer_size=1.5e4*threads_no, batch_size=512, gamma=0.99, tau=1e-3, lr_actor=6e-5, lr_critic=1e-3)

In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Observation space shape: (1, 'Box(10,)')
Action space shape: (1, 'Box(1,)')


### Creating and training agent

In [5]:
# import tensorflow as tf

# class Network(QNetworkTf):
#     def _inference(self):
#         with tf.variable_scope("inference_"+self.name):
#             layer = tf.layers.dense(self.input, 128, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 128, activation=tf.nn.relu)
# #             layer = tf.layers.batch_normalization(layer)
#             layer = tf.layers.dense(layer, 64, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, 32, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 256, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 64, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, self.action_size)
#         return layer

In [ ]:
optimizer = Optimizer("OZwyhJHyqzPZgHEpDFL1zxhyI")
  # Declare your hyper-parameters:
params = """
lr_actor real [1e-7, 1e-4] [6e-5] log
lr_critic real [1e-6, 1e-3] [1e-3] log
"""
optimizer.set_params(params)

agent = Agent(history_length, action_size=1, config=config)
# agent = Agent(Network, history_length, action_size=3, config=config)
# agent.set_epsilon(0.9, 0.01, 25)
teacher = Teacher(agent, env, 1)
# teacher.train(EPISODE_COUNT, simTime, stepTime, "Inp: window Mb sent", "Rew: normalized speed", "DQN", "Convergence", "Instances: 2", "Net: 128, 64, 32")

while True:
    # Get a suggestion
    suggestion = optimizer.get_suggestion()
    config = Config(buffer_size=1.5*threads_no, batch_size=512, gamma=0.99, tau=1e-3, lr_actor=suggestion["lr_actor"], lr_critic=suggestion["lr_critic"])

    # Test the model
    logger = teacher.train(EPISODE_COUNT, simTime, stepTime, "Inp: collisions mb", "Rew: normalized speed", "DDPG", "Convergence", "Actor: 256, 128, 64", "Critic: 512, 256, 128", f"Instances: {threads_no}",
                          **config.__dict__)
    
    # Report the score back
    suggestion.report_score("last_speed", logger.last_speed)


SCRIPT_RUNNING = False

COMET INFO: old comet version (1.0.45) detected. current: 1.0.50 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/c8d98fc7a99c4dc0a81d7b2854e82151

 52%|██████████████████████████████▉                             | 515/1000 [00:45<00:43, 11.20it/s, mb_sent=158.19 Mb]

------- STARTED TRAINING -------


100%|███████████████████████████████████████████████████████████| 1000/1000 [01:37<00:00, 10.23it/s, mb_sent=317.50 Mb]


Sent 317.50 Mb/s.	Mean speed: 31.75 Mb/s	Episode 1/20 finished



100%|███████████████████████████████████████████████████████████| 1000/1000 [01:46<00:00,  9.41it/s, mb_sent=367.84 Mb]


...

100%|███████████████████████████████████████████████████████████| 1000/1000 [01:50<00:00,  9.05it/s, mb_sent=367.82 Mb]


Sent 367.82 Mb/s.	Mean speed: 36.78 Mb/s	Episode 3/20 finished



100%|███████████████████████████████████████████████████████████| 1000/1000 [01:54<00:00,  8.71it/s, mb_sent=344.13 Mb]


Sent 344.13 Mb/s.	Mean speed: 34.41 Mb/s	Episode 4/20 finished



 94%|████████████████████████████████████████████████████████▍   | 941/1000 [01:42<00:06,  9.18it/s, mb_sent=329.46 Mb]

In [ ]:
config.__dict__